## Bayesian Analysis on Credit Card Fraud Detection

Author: Sony Nghiem 

Date: 04/20/2017 

In this project, I searched for a real dataset, studied its features, asked questions, and built statistical models to answer the questions.

### Data

The dataset is composed of European credit card transactions within two days in September, 2013. Out of 284,807 transactions, there are 492 detected frauds.The frauds (positive class in this case) account for .172 % of the whole dataset. The dataset could be retrived through this [link](https://www.kaggle.com/dalpozz/creditcardfraud).

The data contain one binary (0 and 1) variable and 30 features for each observation (or credit card transaction). There is no missing data. Within 30 features included, there are Time (in seconds) and Amount (not clear in unit measurement), and 28 numerical variables, which are principal components of Principal Component Analysis transformation, due to confidentiality issues. 
To have a look at some descriptive statistics about the dataset, please click on this [link](https://github.com/snghiem/creditcardfraud/blob/master/data_visual.ipynb).




### Questions
1. Which models are appropriate for classifying this dataset? (normal vs fraud)
2. Among different statistical that I might build, which one performs better?



### Modeling

To answer the first question, I chose Probit Model using a _Markov chain Monte Carlo_ (MCMC) method, namely _Gibbs Sampling_, to estimate the parameters. I first simulated some data to test the model, and then used the real dataset for this Bayesian analysis. Second, I conducted Naive Bayes model on the dataset. Then I  compared their results in term of F1-score due to the imbalance in data.




#### Probit Model using MCMC

Since a class variable $y$ is binary (0 for normal and 1 for fraud), we consider the following latent variable $z$ representation of the probit model:

$$z_i = x_i \beta + \epsilon_i, \text{ where } \epsilon_i {iid \atop \sim} N(0,1),$$

$$y_i = \Big\{ { \text{ 1 if } z_i>0, \atop \text{ 0 if } z_i\leq 0.} $$

where $x_i$ are the 30 features of each credit card transaction in our dataset (plus one column vector of 1 for the intercept). Both $y_i$ and $x_i$ are observed, while $z$ is an unobserved latent variable.



From Bayes' theory, we have

$$ p(\beta, z |y) \propto p(\beta) p(y,z|\beta)  $$

Compute the _augmented_ data density $p(y,z|\beta)$ as our likelihood and assume a normal distribution for $\beta$ for our prior $p(\beta)$, we obtain the _augmented_ joint posterior:

$$ p(\beta, z |y) \propto p(\beta)\prod_{i=1}^n[I(z_i>0)I(y_i=1) + I(z_i\leq 0)I(y_i=0)]\phi (z_i; x_i\beta, 1) $$ 

Thus, _Gibbs sampling_ could be implemented by sampling iteratively $\beta$ and $z$ as following

$$ \beta|z,y \sim N(D_\beta d_\beta, D_\beta), $$ where
$$D_\beta = (X'X + {V_\beta}^{-1})^{-1}, \text{  }  d_\beta = X'z + {V_\beta}^{-1}\mu_\beta,$$ and

$$ z_i|\beta, y {ind \atop \sim} \Big\{ {TN_{(-\infty, 0]}(x_i\beta, 1) \text{ if } y_i=0, \atop TN_{[0, \infty)}(x_i\beta, 1) \text { if } y_i=1}  $$


Start with a simmulation dataset of 
* $\beta = [-.12, .89, -.72, .50]$
* $\mu_\beta=0$
* $V_\beta = 100I_4$
* $x_i {iid \atop \sim} N(0,1)$
* 10,000 iterations with 1,000 burn-ins (removals)

I got the following result

| True Beta ($\beta$)  | Estimated Beta ($\hat{\beta}$) | Standard Deviation  |
| -------------------- |:------------------------------:| -------------------:|
|     -.12             |       -0.2001601               |   0.06963436        |
|      .89             |        0.95519733              |   0.08798751        |
|     -.72             |       -0.68971489              |   0.07942056        |
|      .50             |        0.41907307              |   0.07524416        |
   
which shows the convergence of $\hat{\beta}$ within two standard deviations. The model gives us 78.4% of correct classifications. For a more detailed result, please see this [link](https://github.com/snghiem/creditcardfraud/blob/master/probit_sim.ipynb).

![Probit Simulation](probit_sim.png). 


![each beta](probit_sim2.png)

Note that the red lines indicate the true betas.

Finally, I applied the model to our real dataset of credit card transactions. [The result gives us 52.57% of correct classifications](https://github.com/snghiem/creditcardfraud/blob/master/probit_fraud.ipynb). 

#### Naive Bayes
[The Naive Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) method applies Bayes' theorem . 

$$ P(Y|X_1,...,X_N) \propto P(Y)P(X_1,...,X_N |Y),$$
where the likelihood
$$P(X_1,...,X_N |Y)= P(X_1|X_2,...,X_N) P(X_2|X_3,...,X_N)...P(X_{N-1}|X_N,Y) P(X_N|Y)$$ 

with a "naive" assumption that all  pair of features ($X_i, X_j$) in data are independent. Thus,

$$ P(X_1,...,X_N |Y)=  \prod_{i=1}^N P(X_i|Y) $$

Consequently, the joint model can be expressed as

$$ P(Y|X_1,...,X_N) \propto P(Y)\prod_{i=1}^N P(X_i|Y) $$

The corresponding Bayes Classifier is then a function that assigns a class label $\hat{Y} = y_k$ as follow
$$ \hat{Y} = {argmax \atop k \in \{1,2,...,K\}} P(Y = y_k) \prod_{i=1}^N P(X_i|Y = y_k)$$

Using _Gaussian Naive Bayes_, which assumes the distribution for each continuous $X_i$ to be of normal distribution, we estimate $\mu_{ik}$ and $\sigma_{ik}^2$ from the likelihood

$$ P(X_i = x| Y = y_k) = \frac{1}{\sqrt{2\pi \sigma_{ik}^2}}exp{\frac{-(x-\mu_{ik})^2}{\sigma_{ik}^2}} $$

For a binary classification, we then have $2n \times 2 +1$ parameters for estimation.

Looking through [the distributions of each feature in our dataset](https://github.com/snghiem/creditcardfraud/blob/master/data_visual.ipynb), I see very low correlation between each individual feature (a correlation heat map is provided). Thus, I assumed each feature is independent from one another and normally distributed.

![correlation](correlation.png)

Using Naive Bayes, I got [97.82% of correct classification](https://github.com/snghiem/creditcardfraud/blob/master/Naive%2BBayes.ipynb).

### Result

To better compare the result from both methods, I decided to do these following:
* Use the same training and testing datasets, which are randomly split $80%$ and $20%$ respectively from the original dataset.
* Use F1-score analysis instead due the high imblance in our original dataset.

For short, [F1-score](https://en.wikipedia.org/wiki/F1_score) (also known as F-measure or balanced F-score) is the harmonic mean of precision ($p$) and recall ($r$), "where $p$ is the number of correct positive results divided by the number of all positive results and $r$ is the number of correct positive results divided by the number of positive results should have been returned".

$$ F_1 = 2 \times \frac{p \times r}{p + r}  $$

Here is a good visual example from [Precision and Recall](https://en.wikipedia.org/wiki/Precision_and_recall):

![https://en.wikipedia.org/wiki/Precision_and_recall#/media/File:Precisionrecall.svg](Precisionrecall.png)

Comparing [the results from two models](https://github.com/snghiem/creditcardfraud/blob/master/model_comparing.ipynb), I got F-1 scores of .095 for the Gaussian Naive Bayes and 0.5156 the Probit model using MCMC.

Since this is only one random sample for both training and testing data. I decided to use bootstraping to get a better look at the range of F-1 scores for each models.

### Conclusion

In summary, I used two models on the dataset of European credit card transactions: _Probit Model_ using _MCMC_ and _Naive Bayes_. To compare their results, I employed bootstrapping and F-1 score analysis due to data imbalance. Though this is not a definite answer, the **** method however shows a better performance for our purpose of binary classification of credit card fraud detection.

### References

* Koop et. al. _Bayesian Econometric Methods_. Cambridge University Press. 2007.
* Mitchell, T. Generative and Discriminative Classifiers: Naive Bayes and Logistic Regression. https://www.cs.cmu.edu/~tom/mlbook/NBayesLogReg.pdf

Project's Link: https://github.com/snghiem/creditcardfraud
